Exercise

Enhance the Employee Listing Program shown in class today to initialize a table from reading a csv file. Then allow a user to
- add a new employee
- delete an employee
- update an employee information
- list all employees
- give 2% raise or $2,000 whichever is greater to all employees
- dump the table content into a csv file before exiting the program

<b>Make sure your inital data consists minimum of 10 employees.</b>

Challenge 1:
- allow a user to specify a criteria for listing employees such as 
    - employees with x salary or more
    - employees started after x year
    - certain employee

Challenge 2: 
- create a department table and incorporate it into your program

In [1]:
import csv                # New Code for this Exercise

def print_list(retval) :  # For printing out lists of employees in neat, tabular form
    for item in retval :
        print(item[0], end="\t")
        if len(item[0]) < 8 :
            print("", end="\t")
        if len(item[0]) < 16 :
            print("", end="\t")
        print(item[1], end="\t")
        if len(item[1]) < 8 :
            print("", end="\t")
        if len(item[1]) < 16 :
            print("", end="\t")
        print(f"Hired {item[2]}\tSalary ${item[3]:,.2f}")

#
# From here to "TASK 1" is all code copied from "Python 2 - Day 4 Class Demo"
#

# importing the sqlite module 
import sqlite3

# pulling the employee.py file and importing the class called Employee - From Day 2 Exercise No. 5 without Customer class
from employee import Employee
### Note:  This isn't used in the current version of the program.

def get_all_employees() :
    """retrieve all employees data from database"""

    # check if db connection exists
    with dbconn:
        db.execute("SELECT * FROM Employees ORDER BY name")
        return db.fetchall()

def get_employee_by_name(name) :
    """retrieve specified employee data from database"""

    # check if db connection exists
    with dbconn:
        db.execute("SELECT * FROM Employees WHERE name = :name", {'name' : name})
        return db.fetchall()

def get_employee_by_salary(salary) :
    """retrieve all employees with greater than or equal to specified salary from database"""

    # check if db connection exists
    with dbconn:
        db.execute("SELECT * FROM Employees WHERE salary >= :salary", {'salary' : salary})
        return db.fetchall()
     
def get_employee_by_year(year) :
    """retrieve all employees started at specified year from database"""

    # check if db connection exists
    with dbconn:
        db.execute("SELECT * FROM Employees WHERE hired_date >= :year", {'year' : year})
# NOTE:  Error in given method.  Field is "hired_date" not "year"
# NOTE:  Changed from = to >= to fit term of problem
        return db.fetchall()

#In memory database
dbconn = sqlite3.connect(':memory:')

# set a variable up for my cursor so I can write SQL scripts
db = dbconn.cursor()

# create Employees table
### Note: This table has no keys.
db.execute("""CREATE TABLE Employees (
            name TEXT,
            address TEXT,
            hired_date INTEGER,
            salary REAL
            )""")

# commits your transactions into the DATABASE
dbconn.commit()

#
# TASK #!: Enhance the Employee Listing Program shown in class today to initialize a table from reading a csv file.
#

with open("employees.csv") as data_file :
    csv_data = csv.reader(data_file)
    for row in csv_data :
        db.execute("INSERT INTO Employees VALUES (?, ?, ?, ?)", 
                    (row[0], row[1], int(row[2]), float(row[3])))
dbconn.commit()

#
# TASK #2: Allow a User Control
#

unsaved = False      # For Save before Quit logic

# Based on Day 2, Exercise 1
print ("MENU")
print ("1. List all employees")
print ("2.    List Specific Employee")
print ("3.    List High Paid Employee")
print ("4.    List Employee hired in Recent Years")
print ("5. Add a new employee")
print ("6. Delete an employee")
print ("7. Update employee information")
print ("8. Give 2% raise (min $2,000) to all employees")
print ("9. Save data")
print ("0. Quit")
print ()

while(True) :
    
    try :
        i = int(input("Enter a menu option: "))
    except :
        print ("Please enter an integer.")
        continue
    
    if (i==1) :
        print("*** All Employees ***")
        retval = get_all_employees()
        print_list(retval)

# NOTE: 2, 3, and 4 are the "Challenge 1" problem.         
    
    elif (i==2) :
        name = input("Who? ")
        retval = get_employee_by_name(name)
        print_list(retval)
    
    elif (i==3) :
        try:
            salary = float(input("Salary: "))
        except:
            print ("ERROR: Salary must be a number")
            continue
        retval = get_employee_by_salary(salary)
        print_list(retval)
    
    elif (i==4) :
        try:
            hired_date = int(input("Hired Date (Year): "))
        except:
            print ("ERROR: Year must be an integer")
            continue
        retval = get_employee_by_year(hired_date)
        print_list(retval)
        
    elif (i==5) :
        name = input("Name: ")
        address = input("Address: ")
        try:
            hired_date = int(input("Hired Date (Year): "))
        except:
            print ("ERROR: Year must be an integer")
            continue
        try:
            salary = float(input("Salary: "))
        except:
            print ("ERROR: Salary must be a number")
            continue
        db.execute("INSERT INTO Employees VALUES (?, ?, ?, ?)", (name, address, hired_date, salary))
        dbconn.commit()
        unsaved = True
    
    elif (i==6) :
        name = input("Delete whom? ")
        db.execute(f"DELETE FROM Employees WHERE name = '{name}'")
            # This is the best we have for a key, but it's not guaranteed to be unique
            # Note: No error is thrown if name is not found
        dbconn.commit()
        unsaved = True

    elif (i==7) :
        name = input("Update whom? ")
        retval = get_employee_by_name(name)
        address = retval[0][1]
        hired_date = retval[0][2]
        salary = retval[0][3]
        
        x = input(f"New Address ({address}): ")
        if x != "" :
            db.execute(f"UPDATE Employees SET address = '{x}' WHERE name = '{name}'")
            dbconn.commit()
            unsaved = True
            print ("updated")
        
        x = input(f"New Hired Date [Year] ({hired_date}): ")
        if x != "" :
            try:
                x = int(x)
            except:
                print ("ERROR: Year must be an integer")
                continue
            db.execute(f"UPDATE Employees SET hired_date = {x} WHERE name = '{name}'")
            dbconn.commit()
            unsaved = True
        
        x = input(f"New Salary (${salary:,.2f}): ")
        if x != "" :
            try:
                x = float(x)
            except:
                print ("ERROR: Salary must be a number")
                continue
            db.execute(f"UPDATE Employees SET salary = {x} WHERE name = '{name}'")
            dbconn.commit()
            unsaved = True
 
    elif (i==8) :
        retval = get_all_employees()
        for item in retval :
            employee_name = item[0]
              # This is the best we have for a key, but it's not guaranteed to be unique
            salary = item[3]
            raise1 = 0.02 * salary
            if (raise1 < 2000) :
                raise1 = 2000
            salary += raise1
            db.execute(f"UPDATE Employees SET salary = {salary} WHERE name = '{employee_name}'")
        dbconn.commit()
        unsaved = True
        print("** DONE **")

    elif (i==9) :
        with open("employees.csv", "w") as data_file :
            csv_data = csv.writer(data_file)
            csv_data.writerows(get_all_employees())
            # Note: Slide 20 of "Python 2 - Day3.pdf" has the calling sequence for this wrong
            #   "writerows" is a method of the writer object, not the data file object
        unsaved = False
        
    elif (i==0) :
        if unsaved :
            j = input("Save before quitting? (y/n) ")
            if j == "n" :
                pass
            elif j == "y" :
                with open("employees.csv", "w") as data_file :
                    csv_data = csv.writer(data_file)
                    csv_data.writerows(get_all_employees())
            else :
                print ('Please enter "y" or "n".')
                continue
        print ("Bye")
        break
        
    else :
        print ("Please enter an integer between 0 and 9.")
    
    print()
     
#
# From here to end is code copied from "Python 2 - Day 4 Class Demo"
#

# good practice to close your connection before exiting your program. 
dbconn.close()


MENU
1. List all employees
2.    List Specific Employee
3.    List High Paid Employee
4.    List Employee hired in Recent Years
5. Add a new employee
6. Delete an employee
7. Update employee information
8. Give 2% raise (min $2,000) to all employees
9. Save data
0. Quit

Enter a menu option: 1
*** All Employees ***
Abby Cadabby		201 Sesame Street	Hired 2006	Salary $50,000.00
Bert			1 Sesame Street		Hired 1969	Salary $60,000.00
Big Bird		123 Sesame Street	Hired 1969	Salary $100,000.00
Cookie Monster		123 Sesame Street	Hired 1969	Salary $50,000.00
Count von Count		129 Sesame Street	Hired 1972	Salary $80,000.00
Elmo			123 Sesame Street	Hired 1980	Salary $50,000.00
Ernie			1 Sesame Street		Hired 1969	Salary $60,000.00
Grover			128 Sesame Street	Hired 1970	Salary $50,000.00
Julia			203 Sesame Street	Hired 2017	Salary $50,000.00
Kermit			123 Sesame Street	Hired 1969	Salary $200,000.00
Oscar the Grouch	131 Sesame Street	Hired 1969	Salary $70,000.00
Rosita			127 Sesame Street	Hired 1991	Salary